In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Data - Original Data.csv", header=1)

# Linearly interpolate population in Scotland
df['SCT_Working_Pop'] = df['SCT_Working_Pop'].interpolate()
df['rUK_Working_Pop'] = df['rUK_Working_Pop'].interpolate()
df['Year'] = df['Year'].fillna(method='ffill')

# Limit sample to 1998Q1 to 2007Q4
# df = df.query("1998 <= Year <= 2007")

df.head(5)

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,UK_Compensation_of_Employees,rUK_Working_Pop,LIBOR,UK_Government_spending,SCT_Government_spending
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.0,61.2685,245778.0,157147.0,111444.0,33910488.0,7.47,38474.0,4267.0
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.0,61.5281,248133.0,158994.0,113808.0,33961408.5,7.49,37534.0,4223.0
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.0,62.1079,251225.0,162342.0,115934.0,34012329.0,7.58,38112.0,4309.0
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.0,62.4503,254190.0,162810.0,116605.0,34063249.5,6.80,39186.0,4299.0
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.0,62.5442,256399.0,165491.0,119895.0,34114170.0,5.51,39760.0,4379.0


In [4]:
for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    df[f'rUK_{var}'] = df[f'UK_{var}'] - df[f'SCT_{var}']

df.head(5)

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,UK_Compensation_of_Employees,rUK_Working_Pop,LIBOR,UK_Government_spending,SCT_Government_spending,rUK_Consumption,rUK_Output,rUK_Compensation_of_Employees,rUK_Government_spending
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.0,61.2685,245778.0,157147.0,111444.0,33910488.0,7.47,38474.0,4267.0,144627.0,226383.0,102000.0,34207.0
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.0,61.5281,248133.0,158994.0,113808.0,33961408.5,7.49,37534.0,4223.0,146359.0,228585.0,104116.0,33311.0
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.0,62.1079,251225.0,162342.0,115934.0,34012329.0,7.58,38112.0,4309.0,149514.0,231757.0,106065.0,33803.0
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.0,62.4503,254190.0,162810.0,116605.0,34063249.5,6.80,39186.0,4299.0,149918.0,234755.0,106684.0,34887.0
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.0,62.5442,256399.0,165491.0,119895.0,34114170.0,5.51,39760.0,4379.0,152417.0,236786.0,109836.0,35381.0


In [5]:
g_y = np.mean(df['SCT_Government_spending'] / df['SCT_Output'])
g_y_ruk = np.mean(df['rUK_Government_spending'] / df['rUK_Output'])

print(f'Government spending-to-output steady state ratio in Scotland is {g_y}')
print(f'Government spending-to-output steady state ratio in the rest of the UK is {g_y_ruk}')

Government spending-to-output steady state ratio in Scotland is 0.25681874740819377
Government spending-to-output steady state ratio in the rest of the UK is 0.1801233751921143


In [29]:
def adjust_series(series, defl, pop):
    return series / (defl * pop)

def diff_log(series):
    return np.log(series).diff()

sct_defl = df["SCT_Deflator (2018=100)"]
sct_pop = df["SCT_Working_Pop"]

adjusted_y = adjust_series(df["SCT_Output"], sct_defl, sct_pop)
diff_log_adjusted_y = diff_log(adjusted_y)
df["y_obs"] = (diff_log_adjusted_y[1:] - np.average(diff_log_adjusted_y[1:])) * 100

adjusted_c = adjust_series(df["SCT_Consumption"], sct_defl, sct_pop)
diff_log_adjusted_c = diff_log(adjusted_c)
df["c_obs"] = (diff_log_adjusted_c[1:] - np.average(diff_log_adjusted_c[1:])) * 100

adjusted_w = adjust_series(df["SCT_Compensation_of_Employees"], sct_defl, sct_pop)
diff_log_adjusted_w = diff_log(adjusted_w)
df["w_obs"] = (diff_log_adjusted_w[1:] - np.average(diff_log_adjusted_w[1:])) * 100

adjusted_g = adjust_series(df["SCT_Government_spending"], sct_defl, sct_pop)
diff_log_adjusted_g = diff_log(adjusted_g)
df["g_obs"] = (diff_log_adjusted_g[1:] - np.average(diff_log_adjusted_g[1:])) * 100

log_pi = np.log(sct_defl)
diff_log_pi = log_pi.diff()
df["pi_obs"] = diff_log_pi[1:] - np.average(diff_log_pi[1:])

diff_interest = df["LIBOR"].diff()
df["i_obs"] = diff_interest[1:]

df

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,rUK_Consumption,rUK_Output,rUK_Compensation_of_Employees,rUK_Government_spending,y_obs,c_obs,w_obs,g_obs,pi_obs,i_obs
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.00,61.2685,245778.0,157147.0,...,144627.0,226383.0,102000.0,34207.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.00,61.5281,248133.0,158994.0,...,146359.0,228585.0,104116.0,33311.0,-0.056766,0.064483,1.485146,-2.256073,-0.002452,0.02
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.00,62.1079,251225.0,162342.0,...,149514.0,231757.0,106065.0,33803.0,-0.639830,1.278892,1.315594,1.409251,-0.008580,0.09
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.00,62.4503,254190.0,162810.0,...,149918.0,234755.0,106684.0,34887.0,-0.245667,0.414333,0.185067,-0.685373,-0.010117,-0.78
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.00,62.5442,256399.0,165491.0,...,152417.0,236786.0,109836.0,35381.0,-0.083618,0.399213,0.121643,0.471456,-0.000924,-1.29
5,1999.0,2.0,65.5,19584.0,13298.0,10350.0,3280784.50,62.7262,258184.0,167521.0,...,154223.0,238600.0,112520.0,35219.0,-0.512675,1.326787,2.222734,-0.536406,-0.007049,-0.31
6,1999.0,3.0,65.9,19812.0,13283.0,10502.0,3280087.00,62.5541,261803.0,168041.0,...,154758.0,241991.0,114593.0,35967.0,0.183961,-1.093713,0.219952,0.299395,-0.000961,-0.01
7,1999.0,4.0,66.3,20148.0,13458.0,10610.0,3279389.50,63.0642,267764.0,169533.0,...,156075.0,247616.0,116262.0,36322.0,0.711880,0.331708,-0.211155,-0.077212,-0.000998,0.71
8,2000.0,1.0,66.4,20383.0,13728.0,10623.0,3278692.00,63.4157,272383.0,172926.0,...,159198.0,252000.0,117535.0,37408.0,0.644212,1.463654,-0.657394,0.054824,-0.005542,0.22
9,2000.0,2.0,66.6,20416.0,13859.0,10886.0,3280680.25,63.4138,274159.0,174426.0,...,160567.0,253743.0,120354.0,38051.0,-0.585569,0.195071,1.433834,2.026532,-0.004042,0.07


In [30]:
def adjust_series(series, defl, pop):
    return series / (defl * 10000 * pop)

def diff_log(series):
    return np.log(series).diff()

rUK_defl = df["UK_Deflator (2023=100)"]
rUK_pop = df["rUK_Working_Pop"]

adjusted_y = adjust_series(df["rUK_Output"], rUK_defl, rUK_pop)
diff_log_adjusted_y = diff_log(adjusted_y)
df["y_obs_ruk"] = (diff_log_adjusted_y[1:] - np.average(diff_log_adjusted_y[1:])) * 100

adjusted_c = adjust_series(df["rUK_Consumption"], rUK_defl, rUK_pop)
diff_log_adjusted_c = diff_log(adjusted_c)
df["c_obs_ruk"] = (diff_log_adjusted_c[1:] - np.average(diff_log_adjusted_c[1:])) * 100

adjusted_w = adjust_series(df["rUK_Compensation_of_Employees"], rUK_defl, rUK_pop)
diff_log_adjusted_w = diff_log(adjusted_w)
df["w_obs_ruk"] = (diff_log_adjusted_w[1:] - np.average(diff_log_adjusted_w[1:])) * 100

adjusted_g = adjust_series(df["rUK_Government_spending"], rUK_defl, rUK_pop)
diff_log_adjusted_g = diff_log(adjusted_g)
df["g_obs_ruk"] = (diff_log_adjusted_g[1:] - np.average(diff_log_adjusted_g[1:])) * 100

log_pi = np.log(rUK_defl)
diff_log_pi = log_pi.diff()
df["pi_obs_ruk"] = diff_log_pi[1:] - np.average(diff_log_pi[1:])

df

,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,c_obs,w_obs,g_obs,pi_obs,i_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,g_obs_ruk,pi_obs_ruk
0,1998.0,1.0,65.1,19395.0,12520.0,9444.0,3281902.00,61.2685,245778.0,157147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,19548.0,12635.0,9692.0,3281797.00,61.5281,248133.0,158994.0,...,0.064483,1.485146,-2.256073,-0.002452,0.02,-0.049790,0.307436,0.769529,-4.168754,-0.001256
2,1998.0,3.0,65.3,19468.0,12828.0,9869.0,3281692.00,62.1079,251225.0,162342.0,...,1.278892,1.315594,1.409251,-0.008580,0.09,-0.154533,0.734857,0.056006,-0.563186,0.003895
3,1998.0,4.0,65.1,19435.0,12892.0,9921.0,3281587.00,62.4503,254190.0,162810.0,...,0.414333,0.185067,-0.685373,-0.010117,-0.78,0.141001,-0.739691,-0.828370,1.515458,0.000014
4,1999.0,1.0,65.5,19613.0,13074.0,10059.0,3281482.00,62.5442,256399.0,165491.0,...,0.399213,0.121643,0.471456,-0.000924,-1.29,0.116898,1.043397,1.901198,0.164817,-0.003982
5,1999.0,2.0,65.5,19584.0,13298.0,10350.0,3280784.50,62.7262,258184.0,167521.0,...,1.326787,2.222734,-0.536406,-0.007049,-0.31,-0.137764,0.411770,1.247349,-1.856574,-0.002578
6,1999.0,3.0,65.9,19812.0,13283.0,10502.0,3280087.00,62.5541,261803.0,168041.0,...,-1.093713,0.219952,0.299395,-0.000961,-0.01,1.075855,0.145717,1.224248,1.269553,-0.008232
7,1999.0,4.0,66.3,20148.0,13458.0,10610.0,3279389.50,63.0642,267764.0,169533.0,...,0.331708,-0.211155,-0.077212,-0.000998,0.71,0.875896,-0.439796,-0.241991,-0.936503,0.002637
8,2000.0,1.0,66.4,20383.0,13728.0,10623.0,3278692.00,63.4157,272383.0,172926.0,...,1.463654,-0.657394,0.054824,-0.005542,0.22,0.589624,0.950601,-0.342358,1.284015,0.000074
9,2000.0,2.0,66.6,20416.0,13859.0,10886.0,3280680.25,63.4138,274159.0,174426.0,...,0.195071,1.433834,2.026532,-0.004042,0.07,0.065328,0.367067,1.480188,0.583608,-0.005514


In [31]:
# Export for dynare

dynare_sct_data_cols = ["y_obs", "c_obs", "w_obs", "pi_obs", "g_obs"]
dynare_ruk_data_cols = ["y_obs_ruk", "c_obs_ruk", "w_obs_ruk", "pi_obs_ruk", "i_obs", "g_obs_ruk"]
dynare_data = df[dynare_sct_data_cols + dynare_ruk_data_cols][1:]
dynare_data.to_csv('../DynareData.csv', index=False)

dynare_data

,y_obs,c_obs,w_obs,pi_obs,g_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,pi_obs_ruk,i_obs,g_obs_ruk
1,-0.056766,0.064483,1.485146,-0.002452,-2.256073,-0.049790,0.307436,0.769529,-0.001256,0.02,-4.168754
2,-0.639830,1.278892,1.315594,-0.008580,1.409251,-0.154533,0.734857,0.056006,0.003895,0.09,-0.563186
3,-0.245667,0.414333,0.185067,-0.010117,-0.685373,0.141001,-0.739691,-0.828370,0.000014,-0.78,1.515458
4,-0.083618,0.399213,0.121643,-0.000924,0.471456,0.116898,1.043397,1.901198,-0.003982,-1.29,0.164817
5,-0.512675,1.326787,2.222734,-0.007049,-0.536406,-0.137764,0.411770,1.247349,-0.002578,-0.31,-1.856574
6,0.183961,-1.093713,0.219952,-0.000961,0.299395,1.075855,0.145717,1.224248,-0.008232,-0.01,1.269553
7,0.711880,0.331708,-0.211155,-0.000998,-0.077212,0.875896,-0.439796,-0.241991,0.002637,0.71,-0.936503
8,0.644212,1.463654,-0.657394,-0.005542,0.054824,0.589624,0.950601,-0.342358,0.000074,0.22,1.284015
9,-0.585569,0.195071,1.433834,-0.004042,2.026532,0.065328,0.367067,1.480188,-0.005514,0.07,0.583608
10,-0.370121,-0.468528,0.442766,0.000430,1.750636,-0.108361,0.299728,0.847673,-0.001132,-0.07,0.176572


In [5]:
import scipy.io as sio
from datetime import date

mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 4:10]
mat_df.columns = ['phi_b', 'phi_g', 'phi_b_f', 'phi_g_f', 'rho_g', 'rho_g_f']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_SCOT_RUK_{date.today()}.csv")
mat_df

,phi_b,phi_g,phi_b_f,phi_g_f,rho_g,rho_g_f
0,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
1,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
2,0.236860,0.075762,0.236873,0.075770,0.924251,0.924251
3,0.315313,0.064457,0.388701,0.089373,0.909613,0.938142
4,0.315313,0.064457,0.388701,0.089373,0.909613,0.938142
...,...,...,...,...,...,...
19995,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19996,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19997,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133
19998,0.225729,0.107164,0.378064,0.066311,0.835557,0.863133


In [3]:
import scipy.io as sio
from datetime import date

mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 2:5]
mat_df.columns = ['phi_b_uk', 'phi_g_uk', 'rho_g_uk']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_UK_{date.today()}.csv")
mat_df

,phi_b_uk,phi_g_uk,rho_g_uk
0,0.004656,0.006227,0.195484
1,0.004656,0.006227,0.195484
2,0.004656,0.006227,0.195484
3,0.004462,0.005090,0.311291
4,0.004462,0.005090,0.311291
...,...,...,...
19995,0.005380,0.004757,0.281375
19996,0.005380,0.004757,0.281375
19997,0.005638,0.006344,0.391866
19998,0.005638,0.006344,0.391866


In [132]:
import scipy.io as sio
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/Output/dissertation_results.mat")['oo_']['irfs']
mat_iter = mat[0][0][0][0]

df = pd.DataFrame()
for var in range(0,len(mat_iter)):
    df[var] = mat_iter[var][0]
df.columns = mat[0][0].dtype.names
df

,debt_eps_g,debt_f_eps_g,b_eps_g,b_f_eps_g,r_real_eps_g,r_real_f_eps_g,g_eps_g,g_f_eps_g,tr_eps_g,tr_f_eps_g,tau_eps_g,tau_f_eps_g,c_eps_g,c_f_eps_g,y_eps_g,y_f_eps_g,wp_eps_g,wp_f_eps_g,n_eps_g,n_f_eps_g
0,0.273601,-5.177624e-17,0.017017,3.173946e-16,0.008539,-1.460399e-17,0.250000,0.000000e+00,-0.023601,5.177624e-17,-0.088142,1.359468e-16,-0.250740,4.532867e-17,-0.000747,4.531587e-17,-0.379139,5.895325e-16,-0.000996,5.688149e-17
1,0.233666,-9.654587e-17,0.238160,1.101437e-15,0.009837,3.130801e-17,0.217500,-2.641846e-18,-0.016166,9.390403e-17,-0.037565,4.068087e-16,-0.218802,4.341137e-17,-0.001308,4.072086e-17,-0.281438,9.586087e-16,-0.001744,5.133231e-17
2,0.199103,-1.090922e-16,0.437529,2.212325e-15,0.008712,5.930678e-17,0.189225,6.260782e-19,-0.009878,1.097182e-16,-0.005485,4.667297e-16,-0.188608,4.495759e-17,0.000612,4.525080e-17,-0.191332,1.098313e-15,0.000816,5.942971e-17
3,0.169160,-1.274695e-16,0.615867,3.648805e-15,0.007534,8.232623e-17,0.164626,7.568996e-18,-0.004534,1.350384e-16,0.020718,5.734978e-16,-0.162186,4.634169e-17,0.002436,5.332287e-17,-0.113926,1.334628e-15,0.003247,7.200532e-17
4,0.143239,-1.514997e-16,0.774564,5.406053e-15,0.006489,1.022134e-16,0.143224,1.674520e-17,-0.000015,1.682449e-16,0.042627,7.206218e-16,-0.139243,4.798160e-17,0.003977,6.390876e-17,-0.048221,1.650266e-15,0.005303,8.772246e-17
5,0.120828,-1.810551e-16,0.914941,7.472434e-15,0.005574,1.193107e-16,0.124605,2.705726e-17,0.003778,2.081124e-16,0.060815,9.015079e-16,-0.119350,5.062329e-17,0.005253,7.665492e-17,0.007136,2.033417e-15,0.007003,1.061285e-16
6,0.101478,-2.158723e-16,1.038214,9.830655e-15,0.004774,1.338047e-16,0.108407,3.766400e-17,0.006929,2.535363e-16,0.075718,1.110215e-15,-0.102114,5.480703e-17,0.006290,9.125873e-17,0.053374,2.473430e-15,0.008387,1.268359e-16
7,0.084799,-2.555515e-16,1.145500,1.245900e-14,0.004075,1.458718e-16,0.094314,4.793166e-17,0.009515,3.034832e-16,0.087718,1.341395e-15,-0.087193,6.085025e-17,0.007118,1.074024e-16,0.091584,2.960185e-15,0.009491,1.494351e-16
8,0.070450,-2.995798e-16,1.237831,1.533228e-14,0.003465,1.556868e-16,0.082053,5.739524e-17,0.011603,3.569750e-16,0.097151,1.590137e-15,-0.074291,6.889701e-17,0.007761,1.247634e-16,0.122727,3.483993e-15,0.010348,1.735093e-16
9,0.058133,-3.473574e-16,1.316170,1.842266e-14,0.002932,1.634192e-16,0.071386,6.572640e-17,0.013253,4.130838e-16,0.104317,1.851868e-15,-0.063146,7.896129e-17,0.008239,1.430264e-16,0.147652,4.035593e-15,0.010986,1.986495e-16


In [8]:
sct_pop_sum = sum(df["SCT_Working_Pop"])
uk_pop_sum = sum(df["rUK_Working_Pop"] + df["SCT_Working_Pop"])

sct_pop_share = sct_pop_sum / uk_pop_sum

sct_pop_share

0.08629149300230622